## Run this notebook

You can launch this notebook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks/air-quality-covid.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Identify available dates and temporal frequency of observations for a given collection - NO2 (Diff)
   2. Pass the STAC item into raster API `/stac/tilejson.json` endpoint
   3. We'll visualize tiles for each of the time steps of interest using `folium`
   

## About the Data

[NO2 (Diff)](https://radiantearth.github.io/stac-browser/#/external/staging-stac.delta-backend.com/collections/no2-monthly-diff)

## Querying the STAC API

In [1]:
import requests
from folium import Map, TileLayer

In [2]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "https://staging-stac.delta-backend.com"
RASTER_API_URL = "https://staging-raster.delta-backend.com"

# Declare collection of interest - Sulfur Dioxide
collection_name = "no2-monthly-diff"

In [3]:
#Fetch STAC collection
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'no2-monthly-diff',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly-diff/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://staging-stac.delta-backend.com/collections/no2-monthly-diff'}],
 'title': 'NO₂ (Diff)',
 'extent': {'spatial': {'bbox': [[-180, -90, 180, 90]]},
  'temporal': {'interval': [['2015-01-01 00:00:00+00',
     '2022-12-01 00:00:00+00']]}},
 'license': 'MIT',
 'summaries': {'datetime': ['2015-01-01T00:00:00Z', '2022-12-01T00:00:00Z'],
  'cog_default': {'max': 23792151724818432, 'min': -20372694184755200}},
 'description': 'This layer shows changes in nitrogen dioxide (NO₂) levels. Redder colors indicate increases in NO₂

Examining the contents of our `collection` under `summaries` we see that the data is available from January 2015 to December 2022. By looking at the `dashboard:time density` we observe that the periodic frequency of these observations is monthly. 

In [4]:
# Check total number of items available
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=100").json()["features"]
print(f"Found {len(items)} items")

Found 96 items


This makes sense as there are 8 years between 2015 - 2022, with 12 months per year, meaning 96 records in total.  

In [5]:
rescale_values = collection["summaries"]["cog_default"]
rescale_values

{'max': 23792151724818432, 'min': -20372694184755200}

## Exploring Changes in Nitrogen Oxide (NO2) Related to Changes in Human Behavior - Using the Raster API

We will explore changes in air quality due to changes in human behaviour resulting from the COVID-19 pandemic. With people largely confined to their homes to reduce the spread of the novel coronavirus, scientists were anticipated there were likely to be fewer cars, planes, and ships emitting fossil fuel pollution. In this notebook, we'll explore the impacts these government lockdowns had on specific air pollutants (i.e., NO2) and explore these changes over time. We'll then visualize the outputs on a map using `folium`. 

In [6]:
# to access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2018-01)
items = {item["properties"]["start_datetime"][:7]: item for item in items} 

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this three times, one for each time step of interest, so that we can visualize each event independently. 

In [8]:
tiles = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2015-05']['collection']}&item={items['2015-05']['id']}"
    "&assets=cog_default"
    "&color_formula=gamma+r+1.05&colormap_name=rdbu_r"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()

# tile_2020 = requests.get(
#     f"{RASTER_API_URL}/stac/tilejson.json?collection={items['2015-05']['collection']}&item={items['2015-05']['id']}"
#     "&assets=cog_default"
#     "&color_formula=gamma+r+1.05&colormap_name=viridis"
#     f"&rescale={rescaling_factor}", 
# ).json()

# tile_2020

## Visualizing Changes in NO2 

In [10]:
# Set initial zoom and map for Galápagos islands

import folium
m = Map(
    tiles="OpenStreetMap", 
    location=[
         40,
         0,
        ], zoom_start=2)

map_layer = TileLayer(
    tiles=tiles["tiles"][0],
    attr="VEDA",
    # opacity=0.6,
)

map_layer.add_to(m)

m

